In [ ]:
# Install required packages
!pip install -q flask nest_asyncio unidecode flask-limiter pytz

import os
import uuid
import subprocess
import time
import nest_asyncio
import threading
import re
from datetime import datetime, timedelta
from flask import Flask, request, render_template_string, redirect, url_for, send_from_directory, jsonify
from werkzeug.utils import secure_filename
from threading import Thread
import locale
import shutil
from werkzeug.exceptions import RequestEntityTooLarge
import pytz

!curl -sL https://tunnelto.dev/install.sh | sh
# Set locale to support Vietnamese
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Get current date in GMT+7 (Bangkok timezone)
gmt7 = pytz.timezone('Asia/Bangkok')
current_date = datetime.now(gmt7).strftime('%Y-%m-%d')

# Default upload folder in Google Drive using current date
DEFAULT_UPLOAD_FOLDER = f'/content/drive/MyDrive/SD-Data/Export/ComfyUI/{current_date}/'

# Create the default folder if it doesn't exist
os.makedirs(DEFAULT_UPLOAD_FOLDER, exist_ok=True)

# Create Flask app
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = DEFAULT_UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 64 * 1024 * 1024  # 16MB max upload size
app.config['MAX_CONTENT_PATH'] = None

# Error handling for file size limit
@app.errorhandler(413)
@app.errorhandler(RequestEntityTooLarge)
def request_entity_too_large(error):
    return redirect(url_for('index', error="File too large! Maximum size is 32MB."))

@app.errorhandler(404)
def not_found(error):
    return redirect(url_for('index', error="Page not found."))

@app.route('/delete_all')
def delete_all_files():
    page = request.args.get('page', 1, type=int)
    try:
        deleted_count = 0
        if os.path.exists(app.config['UPLOAD_FOLDER']):
            for filename in os.listdir(app.config['UPLOAD_FOLDER']):
                file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
                if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.webp')):
                    os.remove(file_path)
                    deleted_count += 1
        
        message = f"Deleted {deleted_count} file(s) successfully"
        return redirect(url_for('index', message=message, page=page))
    except Exception as e:
        error = f"Error deleting files: {str(e)}"
        return redirect(url_for('index', error=error, page=page))

@app.route('/delete_selected', methods=['POST'])
def delete_selected_files():
    page = request.args.get('page', 1, type=int)
    try:
        selected_files = request.form.getlist('selected_files')
        deleted_count = 0
        
        for filename in selected_files:
            file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            if os.path.exists(file_path) and os.path.isfile(file_path):
                os.remove(file_path)
                deleted_count += 1
        
        message = f"Deleted {deleted_count} file(s) successfully"
        return redirect(url_for('index', message=message, page=page))
    except Exception as e:
        error = f"Error deleting files: {str(e)}"
        return redirect(url_for('index', error=error, page=page))


# HTML template for the upload page with Vietnamese support
"""
This is a fixed HTML template for the Upload_image.ipynb file.

To use it:
1. Copy the HTML_TEMPLATE variable below
2. Paste it into your Upload_image.ipynb file, replacing the existing HTML_TEMPLATE variable
"""

HTML_TEMPLATE = '''
<!DOCTYPE html>
<html>
<head>
    <title>Photo Uploader</title>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <style>
        body {
            font-family: Arial, sans-serif;
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
            background-color: #f7f9fc;
        }
        h1, h2 {
            color: #333;
        }
        .container {
            background-color: white;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 0 10px rgba(0,0,0,0.1);
            margin-bottom: 20px;
        }
        .file-list {
            background-color: #f7f7f7;
            padding: 15px;
            border-radius: 5px;
            margin-top: 20px;
        }
        .files-grid {
            display: grid;
            grid-template-columns: repeat(4, 1fr);
            gap: 20px;
            margin-bottom: 20px;
        }
        .file-item {
            display: flex;
            flex-direction: column;
            background-color: white;
            border-radius: 8px;
            overflow: hidden;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
            position: relative;
        }
        .file-checkbox {
            position: absolute;
            top: 10px;
            left: 10px;
            z-index: 10;
            transform: scale(1.5);
            opacity: 0.8;
        }
        .file-actions {
            display: flex;
            justify-content: space-between;
            gap: 5px;
            padding: 8px;
            background-color: #f9f9f9;
        }
        .upload-form {
            margin: 20px 0;
        }
        input[type="text"], input[type="file"] {
            margin-bottom: 10px;
            width: 100%;
            padding: 8px;
            border: 1px solid #ddd;
            border-radius: 4px;
        }
        button, .btn {
            background-color: #4285f4;
            color: white;
            border: none;
            padding: 8px 16px;
            border-radius: 4px;
            cursor: pointer;
            font-size: 14px;
            text-decoration: none;
            display: inline-block;
        }
        button:hover, .btn:hover {
            background-color: #3367d6;
        }
        .btn-delete {
            background-color: #e53935;
        }
        .btn-delete:hover {
            background-color: #c62828;
        }
        .success-message {
            color: #28a745;
            margin-bottom: 15px;
        }
        .error-message {
            color: #dc3545;
            margin-bottom: 15px;
        }
        .folder-name {
            font-weight: bold;
            margin-bottom: 15px;
            word-break: break-all;
        }
        .download-link {
            color: white;
            text-decoration: none;
        }
        .file-info {
            padding: 8px;
            word-break: break-all;
            font-size: 12px;
        }
        .thumbnail-container {
            position: relative;
            width: 100%;
            height: 300px;
            cursor: pointer;
        }
        .file-thumbnail {
            width: 100%;
            height: 100%;
            object-fit: contain;
            background-color: #f0f0f0;
        }
        .refresh-btn {
            margin-left: 10px;
            padding: 4px 8px;
            font-size: 12px;
        }
        .pagination {
            display: flex;
            justify-content: center;
            margin-top: 15px;
            gap: 5px;
        }
        .pagination button {
            min-width: 30px;
        }
        .current-page {
            background-color: #1a73e8;
        }
        .header-actions {
            display: flex;
            align-items: center;
            gap: 10px;
            margin-top: 10px;
            margin-bottom: 10px;
            flex-wrap: wrap;
        }
        .batch-actions {
            margin-top: 10px;
            display: flex;
            gap: 10px;
            align-items: center;
        }
        /* Modal/Lightbox styles */
        .modal {
            display: none;
            position: fixed;
            z-index: 1000;
            padding-top: 50px;
            left: 0;
            top: 0;
            width: 100%;
            height: 100%;
            overflow: auto;
            background-color: rgba(0, 0, 0, 0.9);
        }
        .modal-content {
            margin: auto;
            display: block;
            max-width: 90%;
            max-height: 90%;
        }
        .close {
            position: absolute;
            top: 15px;
            right: 35px;
            color: #f1f1f1;
            font-size: 40px;
            font-weight: bold;
            transition: 0.3s;
            cursor: pointer;
        }
        .close:hover,
        .close:focus {
            color: #bbb;
            text-decoration: none;
        }
        @media (max-width: 1000px) {
            .files-grid {
                grid-template-columns: repeat(3, 1fr);
            }
        }
        @media (max-width: 768px) {
            .files-grid {
                grid-template-columns: repeat(2, 1fr);
            }
        }
        @media (max-width: 480px) {
            .files-grid {
                grid-template-columns: 1fr;
            }
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Upload ảnh cho Comfy</h1>
        <p>Mn download ở đây hoặc download trên drive đều đc, nhưng up ảnh lên/xóa ảnh thì cần thao tác trên này để tránh lỗi<p>
        <p>Mn thao up/xóa trên này xong thì qua Comfy sử dụng như bình thường <p>
        <div class="folder-name">
            Current folder: {{ folder_name }}
        </div>

        {% if success_message %}
        <div class="success-message">
            {{ success_message }}
        </div>
        {% endif %}

        {% if error_message %}
        <div class="error-message">
            {{ error_message }}
        </div>
        {% endif %}

        <form action="/set_folder" method="post" class="upload-form">
            <h2>Change Folder</h2>
            <input type="text" name="folder_name" placeholder="Enter folder name (Vietnamese supported)" value="{{ folder_name }}" required>
            <button type="submit">Set Folder</button>
            <p>Mn chọn nhập tên Folder (trong trường hợp trên drive chưa có Folder sẽ tự tạo folder mới), sau có Click Set Folder<p>
        </form>

        <form action="/upload" method="post" enctype="multipart/form-data" class="upload-form">
            <h2>Upload Photos</h2>
            <input type="file" name="file" multiple accept="image/*" required>
            <button type="submit">Upload</button>
        </form>
        <p>Mọi người upload thì giữ Ctrl + Chọn ảnh để có thể chọn được nhiều ảnh cùng 1 lúc, chọn Upload. Giữ Ctrl+ lăn con lăn chuột để phóng to thumbnail giúp chọn ảnh chuẩn hơn.<p>
    </div>

    <div class="container">
        <h2>
            Uploaded Files
        </h2>
        <p>Lúc mn up ảnh lên ở đây mà không thấy trên drive thì không sao cả, miễn có ảnh ở đây thì Comfy vẫn Denoise/Upscale đc<p>
        <div class="header-actions">
            <button onclick="window.location.href='/?page={{ current_page }}'" class="refresh-btn">Refresh</button>
            {% if files %}
            <a href="/delete_all" class="btn btn-delete" onclick="return confirm('Bạn có chắc muốn xóa TẤT CẢ ảnh trong thư mục này?')">Delete All Photos</a>
            {% endif %}
        </div>
        <p> Click vào thumbnail để xem ảnh phóng to <p>
        
        {% if files %}
        <form id="batch-form" action="/delete_selected" method="post">
            <div class="batch-actions">
                <button type="button" onclick="selectAll()" class="btn">Chọn toàn bộ</button>
                <button type="button" onclick="deselectAll()" class="btn">Bỏ chọn toàn bộ</button>
                <button type="submit" class="btn btn-delete" onclick="return confirmDelete()">Xóa được chọn</button>
            </div>
            
            <div class="file-list">
                <div class="files-grid">
                    {% for file in files %}
                    <div class="file-item">
                        <input type="checkbox" name="selected_files" value="{{ file.name }}" class="file-checkbox">
                        <div class="thumbnail-container" onclick="openModal('{{ file.name }}')">
                            <img src="/thumbnail/{{ file.name }}" class="file-thumbnail" alt="Thumbnail">
                        </div>
                        <div class="file-info">{{ file.name }} ({{ file.size }})<br>{{ file.date }}</div>
                        <div class="file-actions">
                            <a href="/download/{{ file.name }}" class="btn">Download</a>
                            <a href="/delete/{{ file.name }}?page={{ current_page }}" class="btn btn-delete" onclick="return confirm('Bruh có chắc muốn xóa ảnh này?')">Delete</a>
                        </div>
                    </div>
                    {% endfor %}
                </div>

                <div class="pagination">
                    {% if pages > 1 %}
                        {% if current_page > 1 %}
                            <button onclick="window.location.href='/?page={{ current_page - 1 }}'" type="button">←</button>
                        {% endif %}

                        {% for page in range(1, pages + 1) %}
                            <button onclick="window.location.href='/?page={{ page }}'" type="button" {% if page == current_page %}class="current-page"{% endif %}>{{ page }}</button>
                        {% endfor %}

                        {% if current_page < pages %}
                            <button onclick="window.location.href='/?page={{ current_page + 1 }}'" type="button">→</button>
                        {% endif %}
                    {% endif %}
                </div>
            </div>
        </form>
        {% else %}
            <div class="file-list">
                <p>No files in this folder.</p>
            </div>
        {% endif %}
        <p>Mn nên xóa bằng Delete trên trang này, không nên xóa trên Drive nên vì đôi lúc Drive không cập nhật sẽ bị lỗi, VD: Folder De_abc có 50 ảnh và bạn chỉ lấy 10 ảnh để Upscale bằng các xóa trên Drive nhưng vì Drive ko update kịp nên sẽ Upscale hết 50 ảnh * 2' = 100' rất tốn thời gian </p>
    </div>
    <p>Chúc mn sử dụng Comfy vui vẻ ! Trong quá trình sử dụng gặp vấn đề thì liên hệ mình @Đỗ Hưng<p>

    <!-- Modal for image preview -->
    <div id="imageModal" class="modal">
        <span class="close" onclick="closeModal()">&times;</span>
        <img class="modal-content" id="modalImg">
    </div>

    <script>
        // Modal/Lightbox for image preview
        function openModal(imageName) {
            // Don't open modal if clicking on the checkbox
            if (event.target.type === 'checkbox') {
                return;
            }
            
            const modal = document.getElementById('imageModal');
            const modalImg = document.getElementById('modalImg');
            modal.style.display = "block";
            modalImg.src = "/thumbnail/" + imageName;
            
            // Close modal when clicking outside the image
            modal.onclick = function(event) {
                if (event.target === modal || event.target.className === 'close') {
                    closeModal();
                }
            };
        }
        
        function closeModal() {
            document.getElementById('imageModal').style.display = "none";
        }
        
        // Close modal with Escape key
        document.addEventListener('keydown', function(event) {
            if (event.key === "Escape") {
                closeModal();
            }
        });
        
        // Batch selection functions
        function selectAll() {
            const checkboxes = document.querySelectorAll('input[name="selected_files"]');
            checkboxes.forEach(checkbox => {
                checkbox.checked = true;
            });
        }
        
        function deselectAll() {
            const checkboxes = document.querySelectorAll('input[name="selected_files"]');
            checkboxes.forEach(checkbox => {
                checkbox.checked = false;
            });
        }
        
        function confirmDelete() {
            const checkboxes = document.querySelectorAll('input[name="selected_files"]:checked');
            if (checkboxes.length === 0) {
                alert('No files selected!');
                return false;
            }
            return confirm(`Are you sure you want to delete ${checkboxes.length} selected file(s)?`);
        }
        
        // Make thumbnail container clickable, but not when clicking the checkbox
        document.addEventListener('DOMContentLoaded', function() {
            const checkboxes = document.querySelectorAll('.file-checkbox');
            checkboxes.forEach(checkbox => {
                checkbox.addEventListener('click', function(e) {
                    e.stopPropagation();
                });
            });
        });
    </script>
</body>
</html>
'''

# Helper function for Vietnamese folder names
def sanitize_vietnamese_filename(filename):
    """Sanitize filename while preserving Vietnamese characters and slashes"""
    # Only replace illegal filename characters (except slashes)
    illegal_chars = ['\\', ':', '*', '?', '"', '<', '>', '|']
    for char in illegal_chars:
        filename = filename.replace(char, '_')
    return filename.strip()

# Helper function to get human-readable file size
def get_file_size(size_in_bytes):
    for unit in ['B', 'KB', 'MB', 'GB']:
        if size_in_bytes < 1024.0:
            return f"{size_in_bytes:.1f} {unit}"
        size_in_bytes /= 1024.0
    return f"{size_in_bytes:.1f} TB"

# Helper function to get formatted date
def get_formatted_date(timestamp):
    date = datetime.fromtimestamp(timestamp)
    vietnam_time = date + timedelta(hours=7)
    return vietnam_time.strftime("%Y-%m-%d %H:%M:%S")

# Routes
@app.route('/')
def index():
    page = request.args.get('page', 1, type=int)
    per_page = 10  # Number of files per page

    # Get the current folder path and remove the default path from the display
    folder_name_display = app.config['UPLOAD_FOLDER'].replace(f'/content/drive/MyDrive/SD-Data/Export/ComfyUI/{current_date}/', '')

    # Get list of files in the current upload folder
    files = []
    if os.path.exists(app.config['UPLOAD_FOLDER']):
        all_files = []
        for filename in os.listdir(app.config['UPLOAD_FOLDER']):
            file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            if os.path.isfile(file_path):
                # Check if it's an image file
                if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.webp')):
                    size = get_file_size(os.path.getsize(file_path))
                    mod_time = os.path.getmtime(file_path)
                    formatted_date = get_formatted_date(mod_time)
                    all_files.append({
                        'name': filename,
                        'size': size,
                        'date': formatted_date,
                        'timestamp': mod_time
                    })

        # Sort files by modification time (newest first)
        all_files.sort(key=lambda x: x['timestamp'], reverse=True)

        # Calculate pagination
        total_files = len(all_files)
        pages = (total_files + per_page - 1) // per_page  # Ceiling division

        # Adjust page if out of range
        if page < 1:
            page = 1
        elif page > pages and pages > 0:
            page = pages

        # Get files for current page
        start_idx = (page - 1) * per_page
        end_idx = start_idx + per_page
        files = all_files[start_idx:end_idx]

    return render_template_string(
        HTML_TEMPLATE,
        folder_name=folder_name_display,  # Only show relative folder path in the UI
        files=files,
        success_message=request.args.get('message', ''),
        error_message=request.args.get('error', ''),
        current_page=page,
        pages=max(1, (len(files) + per_page - 1) // per_page) if files else 0
    )

@app.route('/set_folder', methods=['POST'])
def set_folder():
    folder_name = request.form.get('folder_name')
    if folder_name:
        # Handle Vietnamese characters properly
        folder_name = sanitize_vietnamese_filename(folder_name)

        # Use absolute path if provided, otherwise assume relative to default
        if not folder_name.startswith('/'):
            folder_name = os.path.join(DEFAULT_UPLOAD_FOLDER, folder_name)

        app.config['UPLOAD_FOLDER'] = folder_name

        try:
            os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)
            message = f"Folder changed to: {app.config['UPLOAD_FOLDER']}"
            return redirect(url_for('index', message=message))
        except Exception as e:
            error = f"Error creating folder: {str(e)}"
            return redirect(url_for('index', error=error))
    else:
        return redirect(url_for('index', error="Please provide a valid folder name"))

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return redirect(url_for('index', error="No file part"))

    files = request.files.getlist('file')

    if not files or files[0].filename == '':
        return redirect(url_for('index', error="No files selected"))

    upload_count = 0
    errors = []

    for file in files:
        if file and file.filename:
            try:
                # Create timestamped filename
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = secure_filename(file.filename)
                base, ext = os.path.splitext(filename)
                new_filename = f"{timestamp}_{base}{ext}"

                # Save the file
                file_path = os.path.join(app.config['UPLOAD_FOLDER'], new_filename)
                file.save(file_path)
                upload_count += 1
            except Exception as e:
                errors.append(f"Error uploading {file.filename}: {str(e)}")

    if errors:
        return redirect(url_for('index', error="\n".join(errors)))
    else:
        message = f"Successfully uploaded {upload_count} file(s)"
        return redirect(url_for('index', message=message))

@app.route('/download/<filename>')
def download_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename, as_attachment=True)

@app.route('/thumbnail/<filename>')
def thumbnail(filename):
    # Simply return the file for now (not an actual thumbnail)
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

@app.route('/delete/<filename>')
def delete_file(filename):
    page = request.args.get('page', 1, type=int)
    try:
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        if os.path.exists(file_path):
            os.remove(file_path)
            message = f"File {filename} deleted successfully"
        else:
            message = f"File {filename} not found"
        return redirect(url_for('index', message=message, page=page))
    except Exception as e:
        error = f"Error deleting file: {str(e)}"
        return redirect(url_for('index', error=error, page=page))

# Function to run Flask app in a separate thread
def run_flask_app(port=5000):
    app.run(host='0.0.0.0', port=port)

# TunnelTo thread function
def tunnelto_thread(port, api):
    import socket
    
    # Wait for the Flask app to start
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    
    # Set the auth key
    cmd = ["/root/.tunnelto/bin/tunnelto", "set-auth", "--key", api[0]]
    subprocess.run(cmd)
    
    # Start the tunnel
    cmd_run = ["/root/.tunnelto/bin/tunnelto", "--subdomain", api[1], "--port", f"{port}"]
    process = subprocess.Popen(cmd_run, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    # Display the tunnel URL
    print(f"\033[92m{'🔗 Link online để sử dụng:'}\033[0m", f"https://{api[1]}.tunn.dev")
    
    # Display additional information
    print("\n============================================================")
    print(f"Photo Uploader is running at: https://{api[1]}.tunn.dev")
    print("This URL can be accessed from any device with internet access")
    print("Upload folder:", app.config['UPLOAD_FOLDER'])
    print("============================================================\n")
    
    return process

# Setup TunnelTo service
def setup_tunnelto_service(port=5000, api=None):
    if api is None:
        # Default TunnelTo credentials
        api = ["68adfdf95e253b06d36e2413d591c81a", "vivivi2upload2222"]
    
    # Start the tunnel in a separate thread
    thread = Thread(target=tunnelto_thread, daemon=True, args=(port, api))
    thread.start()
    return thread

# Main function to set everything up
def setup_photo_uploader(port=5000, tunnelto_api=None):
    """Set up the photo uploader with a TunnelTo public URL"""
    # Ensure Google Drive is mounted
    if not os.path.exists('/content/drive'):
        print("Mounting Google Drive...")
        from google.colab import drive
        drive.mount('/content/drive')

    # Create default folder if it doesn't exist
    os.makedirs(DEFAULT_UPLOAD_FOLDER, exist_ok=True)
    print(f"Default upload folder: {DEFAULT_UPLOAD_FOLDER}")

    # Start Flask in a thread
    flask_thread = Thread(target=run_flask_app, args=(port,))
    flask_thread.daemon = True
    flask_thread.start()

    # Wait for Flask to start
    time.sleep(2)
    
    # Default TunnelTo credentials if not provided
    if tunnelto_api is None:
        tunnelto_api = ["68adfdf95e253b06d36e2413d591c81a", "vivivi2upload2222"]
        print(f"Using default TunnelTo configuration: {tunnelto_api}")
    
    # Set up the TunnelTo tunnel
    tunnelto_thread_obj = setup_tunnelto_service(port=port, api=tunnelto_api)
    
    print("Photo Uploader is starting...")
    print("Wait for the TunnelTo link to appear above")
    
    # Keep the main thread alive
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("Shutting down...")

# Run the setup
if __name__ == "__main__":
    setup_photo_uploader(port=5000)